In [ ]:
# ================================
# 📊 CONFIGURACIÓN AVANZADA Y CARGA DE DATOS
# ================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuración visual profesional
plt.style.use('default')
warnings.filterwarnings('ignore')

# Configuración de visualización mejorada
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

# Paletas de colores profesionales
COLORS = {
    'primary': '#2E86AB',      # Azul principal
    'secondary': '#A23B72',    # Rosa/magenta
    'accent': '#F18F01',       # Naranja
    'success': '#C73E1D',      # Rojo
    'neutral': '#6C757D',      # Gris
    'light': '#F8F9FA',       # Gris claro
    'temporal': ['#0B4D6A', '#2E86AB', '#54C6EB', '#7DD3FC', '#BAE6FD'],  # Gradiente azul
    'heatmap': 'RdYlBu_r',    # Paleta para heatmaps
    'categorical': 'Set2'      # Paleta categórica
}

# Configuración seaborn
sns.set_palette(COLORS['temporal'])
sns.set_style("whitegrid", {"grid.linewidth": 0.1})

print("🚀 SISTEMA DE ANÁLISIS TEMPORAL AVANZADO INICIALIZADO")
print("=" * 70)
print(f"📅 Período de Análisis: 14 Jun 2023 - 31 May 2025 (717 días)")
print(f"🎨 Visualizaciones: Heatmaps • Distribuciones • Correlaciones")
print(f"📈 Granularidades: Diaria • Semanal • Mensual • Trimestral")
print("=" * 70)

def load_and_validate_dataset():
    """Carga y valida el dataset con validaciones temporales robustas"""
    try:
        data_path = Path('data/processed/resultados_pacientes_estandarizados.csv')
        print(f"📂 Cargando dataset desde: {data_path}")
        
        # Cargar dataset completo
        df = pd.read_csv(data_path)
        print(f"✅ Dataset cargado: {len(df):,} registros")
        
        # Conversión temporal robusta
        df['fecha'] = pd.to_datetime(df['fecha'])
        df['fecha_date'] = df['fecha'].dt.date
        
        # Variables temporales adicionales para análisis
        df['año'] = df['fecha'].dt.year
        df['mes'] = df['fecha'].dt.month
        df['dia'] = df['fecha'].dt.day
        df['dia_semana'] = df['fecha'].dt.dayofweek  # 0=Lunes, 6=Domingo
        df['dia_semana_nombre'] = df['fecha'].dt.day_name()
        df['mes_nombre'] = df['fecha'].dt.month_name()
        df['trimestre'] = df['fecha'].dt.quarter
        df['semana_año'] = df['fecha'].dt.isocalendar().week
        df['hora'] = df['fecha'].dt.hour
        
        # Validación temporal
        min_date = df['fecha'].min()
        max_date = df['fecha'].max()
        duration_days = (max_date - min_date).days
        
        print(f"📅 Rango temporal validado:")
        print(f"   • Inicio: {min_date.strftime('%d %B %Y %H:%M')}")
        print(f"   • Fin: {max_date.strftime('%d %B %Y %H:%M')}")
        print(f"   • Duración: {duration_days} días ({duration_days/365.25:.1f} años)")
        
        # Estadísticas por año
        print(f"\n📊 Estadísticas por año:")
        for year in sorted(df['año'].unique()):
            year_data = df[df['año'] == year]
            count = len(year_data)
            cost = year_data['monto_nivel_6'].sum()
            patients = year_data['paciente'].nunique()
            pct = (count / len(df)) * 100
            print(f"   • {year}: {count:,} servicios ({pct:.1f}%) | ${cost:,.0f} | {patients:,} pacientes")
        
        return df
        
    except Exception as e:
        print(f"❌ Error cargando dataset: {e}")
        return None

# Ejecutar carga y validación
df_hospital = load_and_validate_dataset()


In [ ]:
# ====================================
# 🔥 HEATMAPS TEMPORALES AVANZADOS
# ====================================

def create_temporal_heatmaps(df):
    """Crea heatmaps temporales para identificar patrones"""
    print("🔥 CREANDO HEATMAPS TEMPORALES AVANZADOS")
    print("=" * 50)
    
    # Preparar datos agregados
    daily_summary = df.groupby(['año', 'mes', 'dia']).agg({
        'monto_nivel_6': ['count', 'sum'],
        'paciente': 'nunique'
    }).round(2)
    
    daily_summary.columns = ['servicios_count', 'costo_total', 'pacientes_unicos']
    daily_summary = daily_summary.reset_index()
    
    # Crear figura con subplots
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    fig.suptitle('🔥 HEATMAPS TEMPORALES - Patrones de Actividad Hospitalaria (2023-2025)', 
                 fontsize=18, fontweight='bold', y=0.95)
    
    # === HEATMAP 1: DÍA DE SEMANA vs HORA ===
    hourly_weekday = df.groupby(['dia_semana', 'hora'])['monto_nivel_6'].sum().unstack(fill_value=0)
    
    # Crear nombres de días en español
    dia_nombres = ['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sáb', 'Dom']
    hourly_weekday.index = dia_nombres
    
    im1 = axes[0, 0].imshow(hourly_weekday.values, cmap='YlOrRd', aspect='auto', interpolation='bilinear')
    axes[0, 0].set_title('💰 Costos por Día de Semana vs Hora', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Hora del Día')
    axes[0, 0].set_ylabel('Día de la Semana')
    axes[0, 0].set_xticks(range(0, 24, 4))
    axes[0, 0].set_xticklabels([f'{h}:00' for h in range(0, 24, 4)])
    axes[0, 0].set_yticks(range(7))
    axes[0, 0].set_yticklabels(dia_nombres)
    
    # Agregar colorbar
    cbar1 = plt.colorbar(im1, ax=axes[0, 0], fraction=0.046, pad=0.04)
    cbar1.set_label('Costo Total ($)', rotation=270, labelpad=20)
    
    # === HEATMAP 2: MES vs DÍA DEL MES ===
    # Crear pivot table para mes vs día
    monthly_daily = df.groupby(['mes', 'dia'])['monto_nivel_6'].sum().unstack(fill_value=0)
    
    # Asegurar que tenemos todos los días 1-31
    for dia in range(1, 32):
        if dia not in monthly_daily.columns:
            monthly_daily[dia] = 0
    monthly_daily = monthly_daily.reindex(sorted(monthly_daily.columns), axis=1)
    
    im2 = axes[0, 1].imshow(monthly_daily.values, cmap='plasma', aspect='auto', interpolation='bilinear')
    axes[0, 1].set_title('📅 Costos por Mes vs Día del Mes', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Día del Mes')
    axes[0, 1].set_ylabel('Mes')
    axes[0, 1].set_xticks(range(0, 31, 5))
    axes[0, 1].set_xticklabels(range(1, 32, 5))
    axes[0, 1].set_yticks(range(12))
    axes[0, 1].set_yticklabels(['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
                               'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'])
    
    cbar2 = plt.colorbar(im2, ax=axes[0, 1], fraction=0.046, pad=0.04)
    cbar2.set_label('Costo Total ($)', rotation=270, labelpad=20)
    
    # === HEATMAP 3: SEMANA vs AÑO ===
    weekly_yearly = df.groupby(['año', 'semana_año'])['monto_nivel_6'].sum().unstack(fill_value=0)
    
    im3 = axes[1, 0].imshow(weekly_yearly.values, cmap='viridis', aspect='auto', interpolation='bilinear')
    axes[1, 0].set_title('📈 Costos por Año vs Semana del Año', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Semana del Año')
    axes[1, 0].set_ylabel('Año')
    axes[1, 0].set_xticks(range(0, 53, 10))
    axes[1, 0].set_xticklabels(range(1, 54, 10))
    axes[1, 0].set_yticks(range(len(weekly_yearly.index)))
    axes[1, 0].set_yticklabels(weekly_yearly.index)
    
    cbar3 = plt.colorbar(im3, ax=axes[1, 0], fraction=0.046, pad=0.04)
    cbar3.set_label('Costo Total ($)', rotation=270, labelpad=20)
    
    # === HEATMAP 4: TRIMESTRE vs MES ===
    quarterly_monthly = df.groupby(['trimestre', 'mes'])['servicios_count'] = df.groupby(['trimestre', 'mes']).size()
    quarterly_monthly_pivot = quarterly_monthly.unstack(fill_value=0)
    
    im4 = axes[1, 1].imshow(quarterly_monthly_pivot.values, cmap='coolwarm', aspect='auto', interpolation='bilinear')
    axes[1, 1].set_title('🗓️ Servicios por Trimestre vs Mes', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Mes')
    axes[1, 1].set_ylabel('Trimestre')
    axes[1, 1].set_xticks(range(12))
    axes[1, 1].set_xticklabels(['E', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D'])
    axes[1, 1].set_yticks(range(4))
    axes[1, 1].set_yticklabels(['Q1', 'Q2', 'Q3', 'Q4'])
    
    cbar4 = plt.colorbar(im4, ax=axes[1, 1], fraction=0.046, pad=0.04)
    cbar4.set_label('Número de Servicios', rotation=270, labelpad=20)
    
    plt.tight_layout()
    plt.show()
    
    return hourly_weekday, monthly_daily, weekly_yearly

# Ejecutar creación de heatmaps
if df_hospital is not None:
    heatmap_data = create_temporal_heatmaps(df_hospital)


In [ ]:
# ===============================================
# 📊 DISTRIBUCIONES TEMPORALES MEJORADAS
# ===============================================

def create_enhanced_distributions(df):
    """Crea distribuciones temporales mejoradas con múltiples perspectivas"""
    print("📊 CREANDO DISTRIBUCIONES TEMPORALES MEJORADAS")
    print("=" * 55)
    
    # Crear figura grande con múltiples subplots
    fig = plt.figure(figsize=(24, 18))
    
    # Layout de grid personalizado
    gs = fig.add_gridspec(4, 4, hspace=0.3, wspace=0.3)
    
    fig.suptitle('📊 DISTRIBUCIONES TEMPORALES AVANZADAS - Sistema Hospitalario (2023-2025)', 
                 fontsize=20, fontweight='bold', y=0.98)
    
    # === DISTRIBUCIÓN 1: COSTOS DIARIOS (Histograma + KDE) ===
    ax1 = fig.add_subplot(gs[0, :2])
    daily_costs = df.groupby('fecha_date')['monto_nivel_6'].sum()
    
    # Histograma con KDE
    ax1.hist(daily_costs, bins=50, alpha=0.7, color=COLORS['primary'], 
             density=True, edgecolor='white', linewidth=0.5)
    
    # Añadir curva KDE
    from scipy import stats
    kde = stats.gaussian_kde(daily_costs)
    x_range = np.linspace(daily_costs.min(), daily_costs.max(), 200)
    ax1.plot(x_range, kde(x_range), color=COLORS['accent'], linewidth=3, label='Densidad KDE')
    
    ax1.set_title('💰 Distribución de Costos Diarios', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Costo Diario ($)')
    ax1.set_ylabel('Densidad')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Agregar estadísticas
    mean_cost = daily_costs.mean()
    median_cost = daily_costs.median()
    ax1.axvline(mean_cost, color='red', linestyle='--', alpha=0.8, label=f'Media: ${mean_cost:,.0f}')
    ax1.axvline(median_cost, color='green', linestyle='--', alpha=0.8, label=f'Mediana: ${median_cost:,.0f}')
    
    # === DISTRIBUCIÓN 2: SERVICIOS POR DÍA DE SEMANA (Boxplot) ===
    ax2 = fig.add_subplot(gs[0, 2:])
    
    # Preparar datos para boxplot
    weekday_data = []
    weekday_labels = ['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sáb', 'Dom']
    
    for day in range(7):
        day_costs = df[df['dia_semana'] == day].groupby('fecha_date')['monto_nivel_6'].sum()
        weekday_data.append(day_costs.values)
    
    bp = ax2.boxplot(weekday_data, patch_artist=True, labels=weekday_labels)
    
    # Colorear boxes
    colors_week = sns.color_palette("husl", 7)
    for patch, color in zip(bp['boxes'], colors_week):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax2.set_title('📅 Distribución de Costos por Día de Semana', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Costo Diario ($)')
    ax2.grid(True, alpha=0.3)
    ax2.tick_params(axis='x', rotation=45)
    
    # === DISTRIBUCIÓN 3: EVOLUCIÓN MENSUAL (Violin Plot) ===
    ax3 = fig.add_subplot(gs[1, :2])
    
    # Preparar datos mensuales
    monthly_data = []
    month_labels = []
    
    for year in sorted(df['año'].unique()):
        for month in range(1, 13):
            month_data = df[(df['año'] == year) & (df['mes'] == month)]
            if len(month_data) > 0:
                month_costs = month_data.groupby('fecha_date')['monto_nivel_6'].sum()
                monthly_data.append(month_costs.values)
                month_labels.append(f'{year}-{month:02d}')
    
    # Seleccionar cada 3 meses para claridad
    selected_indices = range(0, len(monthly_data), 3)
    selected_data = [monthly_data[i] for i in selected_indices]
    selected_labels = [month_labels[i] for i in selected_indices]
    
    parts = ax3.violinplot(selected_data, positions=range(len(selected_data)), showmeans=True)
    
    # Colorear violins
    for pc, color in zip(parts['bodies'], sns.color_palette("viridis", len(selected_data))):
        pc.set_facecolor(color)
        pc.set_alpha(0.7)
    
    ax3.set_title('🎻 Distribución Mensual de Costos (Violin Plot)', fontsize=14, fontweight='bold')
    ax3.set_ylabel('Costo Diario ($)')
    ax3.set_xticks(range(len(selected_labels)))
    ax3.set_xticklabels(selected_labels, rotation=45)
    ax3.grid(True, alpha=0.3)
    
    # === DISTRIBUCIÓN 4: PACIENTES ÚNICOS POR MES ===
    ax4 = fig.add_subplot(gs[1, 2:])
    
    patients_monthly = df.groupby(['año', 'mes'])['paciente'].nunique().reset_index()
    patients_monthly['año_mes'] = patients_monthly['año'].astype(str) + '-' + patients_monthly['mes'].astype(str).str.zfill(2)
    
    bars = ax4.bar(range(len(patients_monthly)), patients_monthly['paciente'], 
                   color=sns.color_palette("plasma", len(patients_monthly)), alpha=0.8)
    
    ax4.set_title('👥 Pacientes Únicos por Mes', fontsize=14, fontweight='bold')
    ax4.set_ylabel('Pacientes Únicos')
    ax4.set_xticks(range(0, len(patients_monthly), 3))
    ax4.set_xticklabels([patients_monthly.iloc[i]['año_mes'] for i in range(0, len(patients_monthly), 3)], rotation=45)
    ax4.grid(True, alpha=0.3)
    
    # === DISTRIBUCIÓN 5: CORRELACIÓN TEMPORAL (Heatmap) ===
    ax5 = fig.add_subplot(gs[2, :2])
    
    # Crear matriz de correlación temporal
    temporal_metrics = df.groupby('fecha_date').agg({
        'monto_nivel_6': ['count', 'sum', 'mean'],
        'cantidad': 'sum',
        'paciente': 'nunique',
        'area_servicio': 'nunique'
    })
    
    temporal_metrics.columns = ['servicios_count', 'costo_total', 'costo_medio', 
                               'cantidad_total', 'pacientes_unicos', 'areas_distintas']
    
    correlation_matrix = temporal_metrics.corr()
    
    im = ax5.imshow(correlation_matrix, cmap='RdBu_r', vmin=-1, vmax=1, aspect='auto')
    ax5.set_title('🔗 Matriz de Correlación Temporal', fontsize=14, fontweight='bold')
    
    # Configurar ticks
    ax5.set_xticks(range(len(correlation_matrix.columns)))
    ax5.set_yticks(range(len(correlation_matrix.columns)))
    ax5.set_xticklabels(correlation_matrix.columns, rotation=45, ha='right')
    ax5.set_yticklabels(correlation_matrix.columns)
    
    # Añadir valores de correlación
    for i in range(len(correlation_matrix.columns)):
        for j in range(len(correlation_matrix.columns)):
            text = ax5.text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                           ha="center", va="center", color="black", fontweight='bold')
    
    # Colorbar
    cbar = plt.colorbar(im, ax=ax5, fraction=0.046, pad=0.04)
    cbar.set_label('Correlación', rotation=270, labelpad=20)
    
    # === DISTRIBUCIÓN 6: TENDENCIA TEMPORAL POR ÁREA ===
    ax6 = fig.add_subplot(gs[2, 2:])
    
    # Top 5 áreas por costo
    top_areas = df.groupby('area_servicio')['monto_nivel_6'].sum().nlargest(5).index
    
    for i, area in enumerate(top_areas):
        area_monthly = df[df['area_servicio'] == area].groupby(['año', 'mes'])['monto_nivel_6'].sum()
        area_monthly_reset = area_monthly.reset_index()
        area_monthly_reset['periodo'] = range(len(area_monthly_reset))
        
        ax6.plot(area_monthly_reset['periodo'], area_monthly_reset['monto_nivel_6'], 
                marker='o', linewidth=2, label=area[:20], alpha=0.8)
    
    ax6.set_title('📈 Tendencias por Área de Servicio (Top 5)', fontsize=14, fontweight='bold')
    ax6.set_ylabel('Costo Mensual ($)')
    ax6.set_xlabel('Período (Meses)')
    ax6.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax6.grid(True, alpha=0.3)
    
    # === DISTRIBUCIÓN 7: ANÁLISIS ESTACIONAL ===
    ax7 = fig.add_subplot(gs[3, :2])
    
    seasonal_stats = df.groupby('mes').agg({
        'monto_nivel_6': ['mean', 'std'],
        'paciente': 'nunique'
    })
    
    seasonal_stats.columns = ['costo_medio', 'costo_std', 'pacientes_unicos']
    
    x_pos = np.arange(12)
    bars = ax7.bar(x_pos, seasonal_stats['costo_medio'], 
                   yerr=seasonal_stats['costo_std'], 
                   capsize=5, alpha=0.7, color=sns.color_palette("Spectral", 12))
    
    ax7.set_title('🌍 Análisis Estacional - Promedio Mensual ± Std', fontsize=14, fontweight='bold')
    ax7.set_ylabel('Costo Promedio Diario ($)')
    ax7.set_xticks(x_pos)
    ax7.set_xticklabels(['E', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D'])
    ax7.grid(True, alpha=0.3)
    
    # === DISTRIBUCIÓN 8: INTENSIDAD HORARIA ===
    ax8 = fig.add_subplot(gs[3, 2:])
    
    hourly_activity = df.groupby('hora').agg({
        'monto_nivel_6': ['count', 'sum'],
        'paciente': 'nunique'
    })
    
    hourly_activity.columns = ['servicios_count', 'costo_total', 'pacientes_unicos']
    
    # Gráfico de área
    ax8.fill_between(hourly_activity.index, hourly_activity['servicios_count'], 
                     alpha=0.6, color=COLORS['primary'], label='Servicios')
    
    # Eje secundario para costos
    ax8_twin = ax8.twinx()
    ax8_twin.plot(hourly_activity.index, hourly_activity['costo_total'], 
                  color=COLORS['accent'], linewidth=3, marker='o', label='Costos')
    
    ax8.set_title('⏰ Intensidad Horaria - Servicios y Costos', fontsize=14, fontweight='bold')
    ax8.set_xlabel('Hora del Día')
    ax8.set_ylabel('Cantidad de Servicios', color=COLORS['primary'])
    ax8_twin.set_ylabel('Costo Total ($)', color=COLORS['accent'])
    ax8.grid(True, alpha=0.3)
    ax8.legend(loc='upper left')
    ax8_twin.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    return temporal_metrics, correlation_matrix

# Ejecutar creación de distribuciones mejoradas
if df_hospital is not None:
    dist_results = create_enhanced_distributions(df_hospital)


In [ ]:
# ===============================================
# 🎯 HEATMAPS ESPECIALIZADOS POR SERVICIOS
# ===============================================

def create_specialized_service_heatmaps(df):
    """Crea heatmaps especializados por servicios médicos y áreas"""
    print("🎯 CREANDO HEATMAPS ESPECIALIZADOS POR SERVICIOS")
    print("=" * 60)
    
    # Figura con múltiples heatmaps especializados
    fig, axes = plt.subplots(3, 2, figsize=(20, 18))
    fig.suptitle('🎯 HEATMAPS ESPECIALIZADOS - Análisis por Servicios y Áreas (2023-2025)', 
                 fontsize=18, fontweight='bold', y=0.98)
    
    # === HEATMAP 1: TOP 10 SERVICIOS vs MES ===
    # Obtener top 10 servicios por costo total
    top_services = df.groupby('descripcion')['monto_nivel_6'].sum().nlargest(10).index
    
    service_monthly = df[df['descripcion'].isin(top_services)].groupby(['descripcion', 'mes'])['monto_nivel_6'].sum().unstack(fill_value=0)
    
    # Normalizar por fila para ver patrones estacionales
    service_monthly_norm = service_monthly.div(service_monthly.sum(axis=1), axis=0)
    
    im1 = axes[0, 0].imshow(service_monthly_norm.values, cmap='YlOrRd', aspect='auto', interpolation='bilinear')
    axes[0, 0].set_title('💊 Top 10 Servicios vs Mes (Normalizado)', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Mes')
    axes[0, 0].set_ylabel('Servicios Médicos')
    axes[0, 0].set_xticks(range(12))
    axes[0, 0].set_xticklabels(['E', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D'])
    axes[0, 0].set_yticks(range(len(top_services)))
    axes[0, 0].set_yticklabels([service[:25] + '...' if len(service) > 25 else service for service in top_services], fontsize=9)
    
    cbar1 = plt.colorbar(im1, ax=axes[0, 0], fraction=0.046, pad=0.04)
    cbar1.set_label('Proporción del Costo Anual', rotation=270, labelpad=20)
    
    # === HEATMAP 2: ÁREAS DE SERVICIO vs TRIMESTRE ===
    area_quarterly = df.groupby(['area_servicio', 'trimestre'])['monto_nivel_6'].sum().unstack(fill_value=0)
    
    # Normalizar para ver distribución trimestral
    area_quarterly_norm = area_quarterly.div(area_quarterly.sum(axis=1), axis=0)
    
    im2 = axes[0, 1].imshow(area_quarterly_norm.values, cmap='plasma', aspect='auto', interpolation='bilinear')
    axes[0, 1].set_title('🏥 Áreas de Servicio vs Trimestre', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Trimestre')
    axes[0, 1].set_ylabel('Área de Servicio')
    axes[0, 1].set_xticks(range(4))
    axes[0, 1].set_xticklabels(['Q1', 'Q2', 'Q3', 'Q4'])
    axes[0, 1].set_yticks(range(len(area_quarterly.index)))
    axes[0, 1].set_yticklabels([area[:20] + '...' if len(area) > 20 else area for area in area_quarterly.index], fontsize=9)
    
    cbar2 = plt.colorbar(im2, ax=axes[0, 1], fraction=0.046, pad=0.04)
    cbar2.set_label('Proporción Trimestral', rotation=270, labelpad=20)
    
    # === HEATMAP 3: SERVICIOS vs DÍA DE SEMANA ===
    service_weekday = df[df['descripcion'].isin(top_services)].groupby(['descripcion', 'dia_semana'])['monto_nivel_6'].count().unstack(fill_value=0)
    
    # Normalizar por servicio
    service_weekday_norm = service_weekday.div(service_weekday.sum(axis=1), axis=0)
    
    im3 = axes[1, 0].imshow(service_weekday_norm.values, cmap='viridis', aspect='auto', interpolation='bilinear')
    axes[1, 0].set_title('🗓️ Top Servicios vs Día de Semana', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Día de la Semana')
    axes[1, 0].set_ylabel('Servicios Médicos')
    axes[1, 0].set_xticks(range(7))
    axes[1, 0].set_xticklabels(['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sáb', 'Dom'])
    axes[1, 0].set_yticks(range(len(top_services)))
    axes[1, 0].set_yticklabels([service[:25] + '...' if len(service) > 25 else service for service in top_services], fontsize=9)
    
    cbar3 = plt.colorbar(im3, ax=axes[1, 0], fraction=0.046, pad=0.04)
    cbar3.set_label('Proporción Semanal', rotation=270, labelpad=20)
    
    # === HEATMAP 4: INTENSIDAD POR ÁREA vs HORA ===
    top_areas = df.groupby('area_servicio')['monto_nivel_6'].sum().nlargest(8).index
    area_hourly = df[df['area_servicio'].isin(top_areas)].groupby(['area_servicio', 'hora'])['monto_nivel_6'].sum().unstack(fill_value=0)
    
    im4 = axes[1, 1].imshow(area_hourly.values, cmap='hot', aspect='auto', interpolation='bilinear')
    axes[1, 1].set_title('⏰ Top Áreas vs Hora del Día', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Hora del Día')
    axes[1, 1].set_ylabel('Área de Servicio')
    axes[1, 1].set_xticks(range(0, 24, 4))
    axes[1, 1].set_xticklabels([f'{h}:00' for h in range(0, 24, 4)])
    axes[1, 1].set_yticks(range(len(top_areas)))
    axes[1, 1].set_yticklabels([area[:20] + '...' if len(area) > 20 else area for area in top_areas], fontsize=9)
    
    cbar4 = plt.colorbar(im4, ax=axes[1, 1], fraction=0.046, pad=0.04)
    cbar4.set_label('Costo Total ($)', rotation=270, labelpad=20)
    
    # === HEATMAP 5: EVOLUCIÓN ANUAL DE SERVICIOS ===
    service_yearly = df[df['descripcion'].isin(top_services)].groupby(['descripcion', 'año'])['monto_nivel_6'].sum().unstack(fill_value=0)
    
    # Calcular tasa de crecimiento
    if len(service_yearly.columns) > 1:
        growth_rates = service_yearly.pct_change(axis=1).iloc[:, 1:]  # Excluir primera columna (NaN)
        
        im5 = axes[2, 0].imshow(growth_rates.values, cmap='RdBu_r', aspect='auto', 
                               interpolation='bilinear', vmin=-1, vmax=1)
        axes[2, 0].set_title('📈 Tasa de Crecimiento Anual por Servicio', fontsize=14, fontweight='bold')
        axes[2, 0].set_xlabel('Año')
        axes[2, 0].set_ylabel('Servicios Médicos')
        axes[2, 0].set_xticks(range(len(growth_rates.columns)))
        axes[2, 0].set_xticklabels([f'{year-1}→{year}' for year in growth_rates.columns])
        axes[2, 0].set_yticks(range(len(top_services)))
        axes[2, 0].set_yticklabels([service[:25] + '...' if len(service) > 25 else service for service in top_services], fontsize=9)
        
        # Añadir valores de crecimiento
        for i in range(len(top_services)):
            for j in range(len(growth_rates.columns)):
                if not pd.isna(growth_rates.iloc[i, j]):
                    text = axes[2, 0].text(j, i, f'{growth_rates.iloc[i, j]:.1%}',
                                         ha="center", va="center", color="white", fontweight='bold', fontsize=8)
        
        cbar5 = plt.colorbar(im5, ax=axes[2, 0], fraction=0.046, pad=0.04)
        cbar5.set_label('Tasa de Crecimiento', rotation=270, labelpad=20)
    
    # === HEATMAP 6: CORRELACIÓN ENTRE ÁREAS ===
    area_correlation_data = df.groupby(['fecha_date', 'area_servicio'])['monto_nivel_6'].sum().unstack(fill_value=0)
    area_correlation = area_correlation_data.corr()
    
    im6 = axes[2, 1].imshow(area_correlation.values, cmap='coolwarm', aspect='auto', 
                           interpolation='bilinear', vmin=-1, vmax=1)
    axes[2, 1].set_title('🔗 Correlación entre Áreas de Servicio', fontsize=14, fontweight='bold')
    axes[2, 1].set_xlabel('Área de Servicio')
    axes[2, 1].set_ylabel('Área de Servicio')
    axes[2, 1].set_xticks(range(len(area_correlation.columns)))
    axes[2, 1].set_yticks(range(len(area_correlation.columns)))
    axes[2, 1].set_xticklabels([area[:15] + '...' if len(area) > 15 else area for area in area_correlation.columns], 
                              rotation=45, ha='right', fontsize=8)
    axes[2, 1].set_yticklabels([area[:15] + '...' if len(area) > 15 else area for area in area_correlation.columns], fontsize=8)
    
    # Añadir valores de correlación para correlaciones altas
    for i in range(len(area_correlation.columns)):
        for j in range(len(area_correlation.columns)):
            corr_val = area_correlation.iloc[i, j]
            if abs(corr_val) > 0.3 and i != j:  # Solo mostrar correlaciones significativas
                text = axes[2, 1].text(j, i, f'{corr_val:.2f}',
                                     ha="center", va="center", color="black", fontweight='bold', fontsize=7)
    
    cbar6 = plt.colorbar(im6, ax=axes[2, 1], fraction=0.046, pad=0.04)
    cbar6.set_label('Correlación', rotation=270, labelpad=20)
    
    plt.tight_layout()
    plt.show()
    
    return service_monthly_norm, area_quarterly_norm, area_correlation

# Ejecutar creación de heatmaps especializados
if df_hospital is not None:
    specialized_heatmaps = create_specialized_service_heatmaps(df_hospital)


In [ ]:
# ===============================================
# 📈 ANÁLISIS TEMPORAL INTEGRADO CON INSIGHTS
# ===============================================

def create_integrated_temporal_analysis(df):
    """Análisis temporal integrado con insights estadísticos"""
    print("📈 CREANDO ANÁLISIS TEMPORAL INTEGRADO CON INSIGHTS")
    print("=" * 65)
    
    # Crear figura masiva con análisis completo
    fig = plt.figure(figsize=(28, 20))
    gs = fig.add_gridspec(5, 6, hspace=0.35, wspace=0.4)
    
    fig.suptitle('📈 ANÁLISIS TEMPORAL INTEGRADO - Sistema Hospitalario Completo (Jun 2023 - May 2025)', 
                 fontsize=22, fontweight='bold', y=0.98)
    
    # === ANÁLISIS 1: TENDENCIA TEMPORAL PRINCIPAL ===
    ax1 = fig.add_subplot(gs[0, :3])
    
    # Datos diarios agregados
    daily_metrics = df.groupby('fecha_date').agg({
        'monto_nivel_6': ['count', 'sum'],
        'paciente': 'nunique',
        'cantidad': 'sum'
    })
    daily_metrics.columns = ['servicios', 'costos', 'pacientes', 'cantidad']
    
    # Suavizar con rolling window
    daily_smooth = daily_metrics.rolling(window=7, center=True).mean()
    
    # Plot principal
    ax1_twin = ax1.twinx()
    line1 = ax1.plot(daily_smooth.index, daily_smooth['costos'], color=COLORS['primary'], 
                     linewidth=2, alpha=0.8, label='Costos (7d MA)')
    line2 = ax1_twin.plot(daily_smooth.index, daily_smooth['servicios'], color=COLORS['accent'], 
                         linewidth=2, alpha=0.8, label='Servicios (7d MA)')
    
    ax1.set_title('💰 Tendencia Principal: Costos y Servicios (Media Móvil 7 días)', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Costos Diarios ($)', color=COLORS['primary'])
    ax1_twin.set_ylabel('Servicios Diarios', color=COLORS['accent'])
    ax1.grid(True, alpha=0.3)
    
    # Leyenda combinada
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='upper left')
    
    # === ANÁLISIS 2: DECOMPOSICIÓN ESTACIONAL ===
    ax2 = fig.add_subplot(gs[0, 3:])
    
    # Análisis de decomposición estacional simplificado
    monthly_avg = df.groupby('mes')['monto_nivel_6'].mean()
    monthly_std = df.groupby('mes')['monto_nivel_6'].std()
    
    x_months = np.arange(1, 13)
    bars = ax2.bar(x_months, monthly_avg, yerr=monthly_std, capsize=5, 
                   color=sns.color_palette("viridis", 12), alpha=0.7)
    
    # Línea de tendencia
    z = np.polyfit(x_months, monthly_avg, 2)
    p = np.poly1d(z)
    ax2.plot(x_months, p(x_months), "r--", alpha=0.8, linewidth=2, label='Tendencia Cuadrática')
    
    ax2.set_title('🌍 Decomposición Estacional - Patrón Mensual ± Std', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Mes')
    ax2.set_ylabel('Costo Promedio ($)')
    ax2.set_xticks(x_months)
    ax2.set_xticklabels(['E', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D'])
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # === ANÁLISIS 3: HEATMAP CALENDARIO ===
    ax3 = fig.add_subplot(gs[1, :3])
    
    # Crear matriz de calendario
    df_calendar = df.copy()
    df_calendar['year_month'] = df_calendar['fecha'].dt.to_period('M')
    df_calendar['day_of_month'] = df_calendar['fecha'].dt.day
    
    calendar_data = df_calendar.groupby(['year_month', 'day_of_month'])['monto_nivel_6'].sum().unstack(fill_value=0)
    
    # Seleccionar últimos 12 meses para visualización
    recent_months = calendar_data.tail(12)
    
    im3 = ax3.imshow(recent_months.values, cmap='YlOrRd', aspect='auto', interpolation='bilinear')
    ax3.set_title('📅 Heatmap Calendario - Últimos 12 Meses (Costos por Día)', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Día del Mes')
    ax3.set_ylabel('Período (Año-Mes)')
    ax3.set_xticks(range(0, 31, 5))
    ax3.set_xticklabels(range(1, 32, 5))
    ax3.set_yticks(range(len(recent_months.index)))
    ax3.set_yticklabels([str(period) for period in recent_months.index], fontsize=9)
    
    cbar3 = plt.colorbar(im3, ax=ax3, fraction=0.046, pad=0.04)
    cbar3.set_label('Costo Diario ($)', rotation=270, labelpad=20)
    
    # === ANÁLISIS 4: DISTRIBUCIÓN POR CUARTILES ===
    ax4 = fig.add_subplot(gs[1, 3:])
    
    # Calcular cuartiles mensuales
    monthly_quartiles = df.groupby(['año', 'mes'])['monto_nivel_6'].describe()[['25%', '50%', '75%']]
    monthly_quartiles = monthly_quartiles.reset_index()
    monthly_quartiles['periodo'] = range(len(monthly_quartiles))
    
    ax4.fill_between(monthly_quartiles['periodo'], monthly_quartiles['25%'], monthly_quartiles['75%'], 
                     alpha=0.3, color=COLORS['primary'], label='IQR (Q1-Q3)')
    ax4.plot(monthly_quartiles['periodo'], monthly_quartiles['50%'], 
             color=COLORS['primary'], linewidth=3, marker='o', label='Mediana (Q2)')
    
    ax4.set_title('📊 Distribución por Cuartiles - Evolución Mensual', fontsize=14, fontweight='bold')
    ax4.set_xlabel('Período (Meses)')
    ax4.set_ylabel('Costo por Servicio ($)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # === ANÁLISIS 5: TOP SERVICIOS EVOLUTIVOS ===
    ax5 = fig.add_subplot(gs[2, :2])
    
    # Top 5 servicios y su evolución
    top_services = df.groupby('descripcion')['monto_nivel_6'].sum().nlargest(5).index
    
    for i, service in enumerate(top_services):
        service_monthly = df[df['descripcion'] == service].groupby(['año', 'mes'])['monto_nivel_6'].sum()
        service_monthly_reset = service_monthly.reset_index()
        service_monthly_reset['periodo'] = range(len(service_monthly_reset))
        
        ax5.plot(service_monthly_reset['periodo'], service_monthly_reset['monto_nivel_6'], 
                marker='o', linewidth=2, label=service[:20] + '...', alpha=0.8)
    
    ax5.set_title('💊 Evolución Top 5 Servicios Médicos', fontsize=14, fontweight='bold')
    ax5.set_xlabel('Período (Meses)')
    ax5.set_ylabel('Costo Mensual ($)')
    ax5.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax5.grid(True, alpha=0.3)
    
    # === ANÁLISIS 6: CORRELACIÓN TEMPORAL AVANZADA ===
    ax6 = fig.add_subplot(gs[2, 2:4])
    
    # Crear métricas de correlación temporal
    temporal_corr_data = df.groupby('fecha_date').agg({
        'monto_nivel_6': ['count', 'sum', 'mean', 'std'],
        'cantidad': ['sum', 'mean'],
        'paciente': 'nunique'
    })
    
    temporal_corr_data.columns = ['servicios', 'costo_total', 'costo_medio', 'costo_std',
                                 'cantidad_total', 'cantidad_media', 'pacientes_unicos']
    
    # Seleccionar métricas clave
    corr_metrics = temporal_corr_data[['servicios', 'costo_total', 'costo_medio', 
                                      'cantidad_total', 'pacientes_unicos']].corr()
    
    im6 = ax6.imshow(corr_metrics.values, cmap='RdBu_r', vmin=-1, vmax=1, aspect='auto')
    ax6.set_title('🔗 Correlación entre Métricas Temporales', fontsize=14, fontweight='bold')
    
    # Configurar etiquetas
    ax6.set_xticks(range(len(corr_metrics.columns)))
    ax6.set_yticks(range(len(corr_metrics.columns)))
    ax6.set_xticklabels([col.replace('_', ' ').title() for col in corr_metrics.columns], rotation=45, ha='right')
    ax6.set_yticklabels([col.replace('_', ' ').title() for col in corr_metrics.columns])
    
    # Añadir valores de correlación
    for i in range(len(corr_metrics.columns)):
        for j in range(len(corr_metrics.columns)):
            text = ax6.text(j, i, f'{corr_metrics.iloc[i, j]:.2f}',
                           ha="center", va="center", color="white", fontweight='bold')
    
    cbar6 = plt.colorbar(im6, ax=ax6, fraction=0.046, pad=0.04)
    cbar6.set_label('Correlación', rotation=270, labelpad=20)
    
    # === ANÁLISIS 7: VOLATILIDAD TEMPORAL ===
    ax7 = fig.add_subplot(gs[2, 4:])
    
    # Calcular volatilidad móvil
    daily_returns = daily_metrics['costos'].pct_change().dropna()
    volatility = daily_returns.rolling(window=30).std() * np.sqrt(30)  # Volatilidad mensual
    
    ax7.fill_between(volatility.index, volatility, alpha=0.6, color=COLORS['secondary'])
    ax7.plot(volatility.index, volatility, color=COLORS['secondary'], linewidth=2)
    
    ax7.set_title('📈 Volatilidad Temporal (30 días)', fontsize=14, fontweight='bold')
    ax7.set_xlabel('Fecha')
    ax7.set_ylabel('Volatilidad')
    ax7.grid(True, alpha=0.3)
    
    # === ANÁLISIS 8: DISTRIBUCIÓN HORARIA AVANZADA ===
    ax8 = fig.add_subplot(gs[3, :2])
    
    hourly_distribution = df.groupby('hora').agg({
        'monto_nivel_6': ['count', 'sum', 'mean'],
        'paciente': 'nunique'
    })
    hourly_distribution.columns = ['servicios', 'costo_total', 'costo_medio', 'pacientes']
    
    # Crear gráfico radar simplificado como barra polar
    theta = np.linspace(0, 2*np.pi, 24, endpoint=False)
    bars = ax8.bar(theta, hourly_distribution['servicios'], 
                   color=sns.color_palette("hsv", 24), alpha=0.7, width=0.25)
    
    ax8.set_title('⏰ Distribución Horaria de Servicios (Vista Polar)', fontsize=14, fontweight='bold')
    ax8.set_theta_zero_location('N')
    ax8.set_theta_direction(-1)
    ax8.set_thetagrids(range(0, 360, 15), ['12', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
                                          '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'])
    
    # === ANÁLISIS 9: ANÁLISIS DE ÁREAS PRINCIPAL ===
    ax9 = fig.add_subplot(gs[3, 2:4])
    
    area_evolution = df.groupby(['area_servicio', 'mes'])['monto_nivel_6'].sum().unstack(fill_value=0)
    area_evolution_norm = area_evolution.div(area_evolution.sum(axis=1), axis=0)
    
    im9 = ax9.imshow(area_evolution_norm.values, cmap='viridis', aspect='auto', interpolation='bilinear')
    ax9.set_title('🏥 Evolución Estacional por Área (Normalizado)', fontsize=14, fontweight='bold')
    ax9.set_xlabel('Mes')
    ax9.set_ylabel('Área de Servicio')
    ax9.set_xticks(range(12))
    ax9.set_xticklabels(['E', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D'])
    ax9.set_yticks(range(len(area_evolution.index)))
    ax9.set_yticklabels([area[:15] + '...' if len(area) > 15 else area for area in area_evolution.index], fontsize=8)
    
    cbar9 = plt.colorbar(im9, ax=ax9, fraction=0.046, pad=0.04)
    cbar9.set_label('Proporción Mensual', rotation=270, labelpad=20)
    
    # === ANÁLISIS 10: MÉTRICAS DE PERFORMANCE ===
    ax10 = fig.add_subplot(gs[3, 4:])
    
    # KPIs temporales
    performance_metrics = df.groupby(['año', 'mes']).agg({
        'monto_nivel_6': ['sum', 'count'],
        'paciente': 'nunique',
        'cantidad': 'sum'
    })
    performance_metrics.columns = ['costo_total', 'servicios_total', 'pacientes_unicos', 'cantidad_total']
    performance_metrics['costo_por_servicio'] = performance_metrics['costo_total'] / performance_metrics['servicios_total']
    performance_metrics['costo_por_paciente'] = performance_metrics['costo_total'] / performance_metrics['pacientes_unicos']
    
    performance_metrics = performance_metrics.reset_index()
    performance_metrics['periodo'] = range(len(performance_metrics))
    
    # Gráfico doble eje
    ax10_twin = ax10.twinx()
    line1 = ax10.plot(performance_metrics['periodo'], performance_metrics['costo_por_servicio'], 
                      'o-', color=COLORS['primary'], linewidth=2, label='Costo/Servicio')
    line2 = ax10_twin.plot(performance_metrics['periodo'], performance_metrics['costo_por_paciente'], 
                          's-', color=COLORS['accent'], linewidth=2, label='Costo/Paciente')
    
    ax10.set_title('📊 KPIs de Performance Temporal', fontsize=14, fontweight='bold')
    ax10.set_xlabel('Período (Meses)')
    ax10.set_ylabel('Costo por Servicio ($)', color=COLORS['primary'])
    ax10_twin.set_ylabel('Costo por Paciente ($)', color=COLORS['accent'])
    ax10.grid(True, alpha=0.3)
    
    # Leyenda combinada
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax10.legend(lines, labels, loc='upper left')
    
    # === ANÁLISIS 11: RESUMEN ESTADÍSTICO ===
    ax11 = fig.add_subplot(gs[4, :3])
    
    # Tabla de estadísticas clave
    stats_summary = df.groupby('año').agg({
        'monto_nivel_6': ['count', 'sum', 'mean', 'std'],
        'paciente': 'nunique',
        'area_servicio': 'nunique',
        'descripcion': 'nunique'
    }).round(2)
    
    stats_summary.columns = ['Servicios', 'Costo Total', 'Costo Medio', 'Costo Std', 
                            'Pacientes', 'Áreas', 'Descripciones']
    
    # Crear tabla como heatmap
    stats_normalized = stats_summary.div(stats_summary.max())
    im11 = ax11.imshow(stats_normalized.values, cmap='Blues', aspect='auto')
    
    ax11.set_title('📋 Resumen Estadístico Anual (Normalizado)', fontsize=14, fontweight='bold')
    ax11.set_xticks(range(len(stats_summary.columns)))
    ax11.set_yticks(range(len(stats_summary.index)))
    ax11.set_xticklabels(stats_summary.columns, rotation=45, ha='right')
    ax11.set_yticklabels(stats_summary.index)
    
    # Añadir valores reales
    for i in range(len(stats_summary.index)):
        for j in range(len(stats_summary.columns)):
            text = ax11.text(j, i, f'{stats_summary.iloc[i, j]:,.0f}',
                            ha="center", va="center", color="black", fontweight='bold', fontsize=9)
    
    # === ANÁLISIS 12: PREDICCIÓN BÁSICA ===
    ax12 = fig.add_subplot(gs[4, 3:])
    
    # Predicción simple usando tendencia lineal
    monthly_totals = df.groupby(['año', 'mes'])['monto_nivel_6'].sum().reset_index()
    monthly_totals['periodo'] = range(len(monthly_totals))
    
    # Ajuste lineal
    z = np.polyfit(monthly_totals['periodo'], monthly_totals['monto_nivel_6'], 1)
    p = np.poly1d(z)
    
    # Predicción para próximos 6 meses
    future_periods = np.arange(len(monthly_totals), len(monthly_totals) + 6)
    future_predictions = p(future_periods)
    
    # Plot
    ax12.plot(monthly_totals['periodo'], monthly_totals['monto_nivel_6'], 
              'o-', color=COLORS['primary'], linewidth=2, label='Datos Históricos')
    ax12.plot(monthly_totals['periodo'], p(monthly_totals['periodo']), 
              '--', color=COLORS['secondary'], linewidth=2, label='Tendencia')
    ax12.plot(future_periods, future_predictions, 
              's-', color=COLORS['accent'], linewidth=2, label='Predicción 6M')
    
    ax12.set_title('🔮 Predicción Básica - Próximos 6 Meses', fontsize=14, fontweight='bold')
    ax12.set_xlabel('Período (Meses)')
    ax12.set_ylabel('Costo Mensual Total ($)')
    ax12.legend()
    ax12.grid(True, alpha=0.3)
    
    # Sombrear área de predicción
    ax12.axvspan(len(monthly_totals), len(monthly_totals) + 6, alpha=0.2, color=COLORS['accent'])
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 RESUMEN DE INSIGHTS CLAVE:")
    print("=" * 50)
    print(f"• Total de días analizados: {len(daily_metrics)}")
    print(f"• Promedio diario de costos: ${daily_metrics['costos'].mean():,.0f}")
    print(f"• Volatilidad promedio: {volatility.mean():.2%}")
    print(f"• Servicios más costosos identificados: {len(top_services)}")
    print(f"• Correlación costos-servicios: {corr_metrics.loc['servicios', 'costo_total']:.2f}")
    
    return daily_metrics, volatility, performance_metrics

# Ejecutar análisis temporal integrado
if df_hospital is not None:
    integrated_results = create_integrated_temporal_analysis(df_hospital)
